# Promptfoo: classification evaluations

**Note: このレッスンは関連コードファイルを含むフォルダ内にあります。実際に手元で実行しながら進めたい場合は、フォルダ全体をダウンロードしてください。**

以前のレッスンでは、次のような顧客の苦情：

> Whenever I open your app, my phone gets really slow

や

> I can't figure out how to change my password

を、次の5カテゴリに分類するプロンプトを評価しました：
- Software Bug
- Hardware Malfunction
- User Error
- Service Outage
- Feature Request

このレッスンでは、そのプロンプト評価を promptfoo に移植し、まとめて実行（バッチ）したり結果を比較したりしやすくします。

---


## Initializing promptfoo

最初に、次のコマンドで promptfoo を初期化します：

```bash
npx promptfoo@latest init
```

前のレッスンと同様に、これで `promptfooconfig.yaml` が作成されます。既存の内容は削除して構いません。

次にプロバイダを設定します。`promptfooconfig.yaml` に次を追加してください：

```yaml
description: "Complaint Classification Eval"
  
providers:
  - "anthropic:messages:claude-3-haiku-20240307"
```

このレッスンでは評価を何度も回すので、APIコストを抑えるため Claude 3 Haiku を使います。

**`ANTHROPIC_API_KEY` 環境変数が設定されていることを確認してください。ターミナルで次のように設定できます：**

```bash
export ANTHROPIC_API_KEY=your_api_key_here
```

---


## Preparing our prompts

次に、プロンプトをまとめて promptfoo に認識させます。前の動画（前レッスン）と同じパターンで進めます：

- 各プロンプトを Python 関数として定義する
- 各関数はプロンプト文字列を返す
- すべてのプロンプト関数を `prompts.py` に入れる

`prompts.py` を新規作成し、次のプロンプト関数を追加してください。これは元の「苦情分類」レッスンで作った2つのプロンプトと同じです：

```py
def basic_prompt(complaint):
    return f"""
    次の顧客の苦情を、以下のカテゴリの1つ以上に分類してください： 
    Software Bug、Hardware Malfunction、User Error、Feature Request、または Service Outage。
    分類のみを回答してください。

    苦情: {complaint}

    分類:
    """

def improved_prompt(complaint):
    return f"""
    あなたは顧客サポートの問題分類を専門とするAIアシスタントです。顧客の苦情を分析し、以下のカテゴリの1つ以上に分類するのがあなたの仕事です：

    1. Software Bug: ソフトウェアが意図通りに機能しない問題。
    2. Hardware Malfunction: 物理的なデバイスやコンポーネントの問題。
    3. User Error: ユーザーの誤解や誤用による困難。
    4. Feature Request: 新機能や改善の提案。
    5. Service Outage: サービスの可用性に影響を与えるシステム全体の問題。

    重要なガイドライン：
    - 苦情は複数のカテゴリに該当する場合があります。その場合は、該当するすべてをリストアップしてください。ただし、可能な限り単一のカテゴリを優先的に選択するよう努めてください。

    例：
    1. 苦情: "アプリが進捗を保存しようとするたびにクラッシュします。"
    分類: Software Bug

    2. 苦情: "コーヒーをこぼした後、キーボードが動作しなくなりました。"
    分類: Hardware Malfunction

    3. 苦情: "ウェブサイトでログインボタンが見つかりません。"
    分類: User Error

    4. 苦情: "アプリにダークモードがあると素晴らしいです。"
    分類: Feature Request

    5. 苦情: "私や同僚にとって、あなたのサービスのどれも読み込まれません。"
    分類: Service Outage

    6. 苦情: "アプリがプロフィール写真を変更しようとするたびに壊れます"
    分類: Software Bug

    7. 苦情: "アプリが私の電話でバグだらけで、ウェブサイトもダウンしているようで、完全に行き詰まっています！"
    分類: Software Bug, Service Outage

    8. 苦情: "あなたのソフトウェアが私のコンピューターを非常に遅くし、最悪です、嫌いです！"
    分類: Software Bug

    9. 苦情: "あなたのダメなアプリは、画像で何かしようとするたびに常に壊れます。"
    分類: Software Bug

    それでは、次の顧客の苦情を分類してください：

    <complaint>{complaint}</complaint>

    適切なカテゴリのみを回答し、それ以外は何も含めないでください。
    分類:
    """
```

次に、promptfoo にこの2つのプロンプトを使うことを伝えます。`promptfooconfig.yaml` を次のように更新してください：

```yaml
description: "Complaint Classification Eval"

prompts:
  - prompts.py:basic_prompt
  - prompts.py:improved_prompt
  
providers:
  - "anthropic:messages:claude-3-haiku-20240307"
```

---


## Preparing our evaluation test set

最後のステップは、評価データセットを promptfoo と相性の良い形にすることです。復習として、以前のレッスンの `eval_data` は次のような Python リストでした：

```py
eval_data = [
    {
        "complaint": "The app crashes every time I try to upload a photo",
        "golden_answer": ["Software Bug"]
    },
    {
        "complaint": "My printer isn't recognized by my computer",
        "golden_answer": ["Hardware Malfunction"]
    },
    {
        "complaint": "I can't figure out how to change my password",
        "golden_answer": ["User Error"]
    },
    {
        "complaint": "The website is completely down, I can't access any pages",
        "golden_answer": ["Service Outage"]
    },
    {
        "complaint": "It would be great if the app had a dark mode option",
        "golden_answer": ["Feature Request"]
    },
    {
        "complaint": "The software keeps freezing when I try to save large files",
        "golden_answer": ["Software Bug"]
    },
    {
        "complaint": "My wireless mouse isn't working, even with new batteries",
        "golden_answer": ["Hardware Malfunction"]
    },
    {
        "complaint": "I accidentally deleted some important files, can you help me recover them?",
        "golden_answer": ["User Error"]
    },
    {
        "complaint": "None of your servers are responding, is there an outage?",
        "golden_answer": ["Service Outage"]
    },
    {
        "complaint": "Could you add a feature to export data in CSV format?",
        "golden_answer": ["Feature Request"]
    },
    {
        "complaint": "The app is crashing and my phone is overheating",
        "golden_answer": ["Software Bug", "Hardware Malfunction"]
    },
    {
        "complaint": "I can't remember my password!",
        "golden_answer": ["User Error"]
    },
    {
        "complaint": "The new update broke something and the app no longer works for me",
        "golden_answer": ["Software Bug"]
    },
    {
        "complaint": "I think I installed something incorrectly, now my computer won't start at all",
        "golden_answer": ["User Error", "Hardware Malfunction"]
    },
    {
        "complaint": "Your service is down, and I urgently need a feature to batch process files",
        "golden_answer": ["Service Outage", "Feature Request"]
    },
    {
        "complaint": "The graphics card is making weird noises",
        "golden_answer": ["Hardware Malfunction"]
    },
    {
        "complaint": "My keyboard just totally stopped working out of nowhere",
        "golden_answer": ["Hardware Malfunction"]
    },
    {
        "complaint": "Whenever I open your app, my phone gets really slow",
        "golden_answer": ["Software Bug"]
    },
    {
        "complaint": "Can you make the interface more user-friendly? I always get lost in the menus",
        "golden_answer": ["Feature Request", "User Error"]
    },
    {
        "complaint": "The cloud storage isn't syncing and I can't access my files from other devices",
        "golden_answer": ["Software Bug", "Service Outage"]
    }
]
```


前のレッスンと同様に、このデータセットを CSV にします。ただし今回は、評価ロジックが単純な完全一致（exact-match）ではありません。採点するには、promptfoo が「各モデル出力に正しい分類が含まれていること」を確認できる必要があります。

たとえば次の行：

```py
{
    "complaint": "The cloud storage isn't syncing and I can't access my files from other devices",
    "golden_answer": ["Software Bug", "Service Outage"]
}
```

この入力 `complaint` をプロンプトに渡すと：

> The cloud storage isn't syncing and I can't access my files from other devices

出力には "Software Bug" と "Service Outage" の両方が含まれている必要があります。ここでは完全一致は使えません（順序が入れ替わる可能性もあります）。幸い、promptfoo には多くの組み込みアサーションがあります。例えば：

* `contains` - 出力が部分文字列を含む
* `contains-all` - 出力がリスト内の全ての部分文字列を含む
* `contains-any` - 出力がリスト内のいずれかの部分文字列を含む
* `contains-json` - 出力が妥当な JSON（任意でJSON Schema検証）
* `contains-sql` - 出力が妥当な SQL
* `contains-xml` - 出力が妥当な XML
* `equals` - 出力が完全一致
* `icontains` - 大文字小文字を無視した `contains`
* `icontains-all` - 大文字小文字を無視した `contains-all`
* `icontains-any` - 大文字小文字を無視した `contains-any`
* `regex` - 正規表現に一致
* ほか多数

[組み込みメトリクスの一覧はこちら](https://www.promptfoo.dev/docs/configuration/expected-outputs/deterministic/)

今回のユースケースでは `contains-all` を使って、必要な分類ラベルがすべて含まれていることを保証します。

promptfoo の評価データセットは CSV でも定義できます。前回と同様、`__expected` という特別な列を使って採点ロジックを指定できます。この列では、上記の組み込みアサーション（例：`contains-all`）を利用できます。


`dataset.csv` という名前の新しいファイルを作成し、次のコードを貼り付けてください： 

```csv
complaint,__expected
The app crashes every time I try to upload a photo,contains-all:Software Bug
My printer isn't recognized by my computer,contains-all:Hardware Malfunction
I can't figure out how to change my password,contains-all:User Error
The website is completely down I can't access any pages,contains-all:Service Outage
It would be great if the app had a dark mode option,contains-all:Feature Request
The software keeps freezing when I try to save large files,contains-all:Software Bug
My wireless mouse isn't working even with new batteries,contains-all:Hardware Malfunction
I accidentally deleted some important files can you help me recover them?,contains-all:User Error
None of your servers are responding is there an outage?,contains-all:Service Outage
Could you add a feature to export data in CSV format?,contains-all:Feature Request
"The app is crashing and my phone is overheating","contains-all:Software Bug,Hardware Malfunction"
I can't remember my password!,contains-all:User Error
The new update broke something and the app no longer works for me,contains-all:Software Bug
"I think I installed something incorrectly now my computer won't start at all","contains-all:User Error,Hardware Malfunction"
"Your service is down and I urgently need a feature to batch process files","contains-all:Service Outage,Feature Request"
The graphics card is making weird noises,contains-all:Hardware Malfunction
My keyboard just totally stopped working out of nowhere,contains-all:Hardware Malfunction
Whenever I open your app my phone gets really slow,contains-all:Software Bug
Can you make the interface more user-friendly? I always get lost in the menus,"contains-all:Feature Request,User Error"
The cloud storage isn't syncing and I can't access my files from other devices,"contains-all:Software Bug,Service Outage"
```

このCSVは2列あります：
- `complaint`：入力の苦情文
- `__expected`：`contains-all` アサーション

例えば次の行：

> "Your service is down and I urgently need a feature to batch process files","contains-all:Service Outage,Feature Request"

は、入力が "Your service is down and I urgently need a feature to batch process files" のとき、モデル出力に "Service Outage" と "Feature Request" の両方が含まれていることを求めています。

最後に、書いたテストを `promptfooconfig.yaml` に追加します。ファイルは次のようになります：

```yaml
description: "Complaint Classification Eval"

prompts:
  - prompts.py:basic_prompt
  - prompts.py:improved_prompt
  
providers:
  - "anthropic:messages:claude-3-haiku-20240307"

tests: dataset.csv

```

---


## Running the evaluation

評価を実行するには、これまでと同じコマンドを使います：

```bash
npx promptfoo@latest eval
```


これは、最初に評価を実行したときの出力例です：

![eval_output.png](../images/eval_output.png)

例を含む改善プロンプトの方が、初期の basic プロンプトより良い性能になっているのが分かります。例があることで、複数カテゴリにまたがるケースをモデルが理解しやすくなります：

![output_row.png](../images/output_row.png)


いつも通り、次のコマンドで評価結果のWebビューも開けます：

```bash
npx promptfoo@latest view
```

![web_view.png](../images/web_view.png)


`basic_prompt` は 80% 正解で、`improved_prompt` は 100% でした。

**繰り返しになりますが、このコースでは教育目的の小さなデータセットを使っています。実運用の評価では、少なくとも100行以上の評価データセットを推奨します。**

次は、promptfoo でカスタム採点ロジック（custom grading logic）を書く方法を見ていきます！
